# MILP Formulation from Nathan Kallus' Paper (Problem 4)

In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys
import math
import numpy as np
import pandas as pd
import random

## Model specifications
#### Since we chose to modify the formulation to a certain extent, these variables simply allow us to revert back to the original model
- delta_include: If true, then constraint 4c from original formulation holds. If false, only the added constraint that gamma[p] need to add to 1 holds
- different_Cp = If true, then we have different sets of branching choices for every non-leaf node (like original formulation). If false, then we have a static set C for all nodes

In [2]:
delta_include = False
different_Cp = False

## Generating Synthetic Data
INPUT:
- n = number of data points

OUTPUT:
- Pandas DataFrame (nx12)
    - first 10 rows: covariates (first 5 useful, next 5 is noise)
    - row 11: choice of treatment
        Determined by imposing a probability on the covariates
    - row 12: outcome based on treatment
        Samples from a uniform distribution. If the expected treatment was given, then the outcome would be lower (i.e. better)

In [3]:
def generate_data(n):
    # initialize df of n x 12
    #columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'T', 'Y']
    data = pd.DataFrame()
    for i in range(n):
        # Generate useful variables (5 binary numbers)
        useful = np.random.randint(low=0, high=2, size=5) # high is exclusive
        
        # Generate noise variables
        noise = np.random.randint(low=0, high=2, size=5)
        
        # Determine the probability of treatment given the useful variables
        # Generate outcome depending on the desired value: we assume that lower outcome = better
        
        # Linear transformation of useful variables
        pi = 2*useful[0] + useful[1] - 3*useful[2] + 2*useful[3] + 3*useful[4] - 2
        probability = np.exp(pi)/(1+ np.exp(pi))
        if probability >= 0.5:
            t = np.random.binomial(n=1, p=0.9)
            if t == 1:
                y = np.random.uniform(0, 0.6)
            else:
                y = np.random.uniform(0.45, 1)
        else:
            t = np.random.binomial(n=1, p=0.2)
            if t == 0:
                y = np.random.uniform(0, 0.65)
            else:
                y = np.random.uniform(0.5, 1)

        row = pd.DataFrame(np.concatenate((useful, noise, t, y), axis=None).reshape(1, 12))
        
        data = data.append(row, ignore_index=True)
    
    return data

In [3]:
def generate_data2(n):
    # initialize df of n x 12
    #columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'T', 'Y']
    data = pd.DataFrame()
    for i in range(n):
        # Generate useful variables (5 binary numbers)
        useful = np.random.randint(low=0, high=2, size=1) # high is exclusive
        
        # Generate noise variables
        noise = np.random.randint(low=0, high=2, size=1)
        
        # Determine the probability of treatment given the useful variables
        # Generate outcome depending on the desired value: we assume that lower outcome = better
        
        if useful == 0:
            t = np.random.binomial(n=1, p=0.1)
            if t == 0:
                y = 0
            else:
                y = 0.2
        else:
            t = np.random.binomial(n=1, p=0.9)
            if t == 0:
                y = 1
            else:
                y = 0.8

        row = pd.DataFrame(np.concatenate((useful, noise, t, y), axis=None).reshape(1, 4))
        
        data = data.append(row, ignore_index=True)
    
    return data

In [4]:
# --- OLD SYNTHETIC DATA (using a more complicated distribution)

"""def generate_data(n):
    # initialize df of n x 12
    #columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'T', 'Y']
    data = pd.DataFrame()
    for i in range(n):
        # Generate useful variables
        useful = np.concatenate((np.random.normal(0, 1, 3), np.random.uniform(0, 1, 2)), axis=None)
        
        # Generate noise variables
        noise = np.concatenate((np.random.normal(5, 2, 2), np.random.uniform(6, 30, 2), np.random.exponential(0.5)), axis=None)
        
        # Determine the probability of treatment given the useful variables
        # Generate outcome depending on the desired value: we assume that lower outcome = better
        if (useful[0] > 0.2 and useful[1] < 0) or (useful[2] > 0.3 and useful[3] > 0.7) or (useful[4] < 0.2 and useful[1] > 0.5):
            t = np.random.binomial(n=1, p=0.9)
            if t == 1:
                y = np.random.uniform(0, 0.6)
            else:
                y = np.random.uniform(0.45, 1)
        else:
            t = np.random.binomial(n=1, p=0.2)
            if t == 0:
                y = np.random.uniform(0, 0.65)
            else:
                y = np.random.uniform(0.4, 1)
                
        row = pd.DataFrame(np.concatenate((useful, noise, t, y), axis=None).reshape(1, 12))
        #print(row)
        
        data = data.append(row, ignore_index=True)
    
    return data"""

"def generate_data(n):\n    # initialize df of n x 12\n    #columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'T', 'Y']\n    data = pd.DataFrame()\n    for i in range(n):\n        # Generate useful variables\n        useful = np.concatenate((np.random.normal(0, 1, 3), np.random.uniform(0, 1, 2)), axis=None)\n        \n        # Generate noise variables\n        noise = np.concatenate((np.random.normal(5, 2, 2), np.random.uniform(6, 30, 2), np.random.exponential(0.5)), axis=None)\n        \n        # Determine the probability of treatment given the useful variables\n        # Generate outcome depending on the desired value: we assume that lower outcome = better\n        if (useful[0] > 0.2 and useful[1] < 0) or (useful[2] > 0.3 and useful[3] > 0.7) or (useful[4] < 0.2 and useful[1] > 0.5):\n            t = np.random.binomial(n=1, p=0.9)\n            if t == 1:\n                y = np.random.uniform(0, 0.6)\n            else:\n                y = np.random.uniform(0.4

In [14]:
np.random.seed(1)

data = generate_data2(100)
treatment = data.iloc[:, 2].to_numpy()
outcome = data.iloc[:,3].to_numpy()
data = data.iloc[:, :2]
print(data)
print(outcome)
print(treatment)

      0    1
0   1.0  1.0
1   1.0  1.0
2   1.0  0.0
3   0.0  1.0
4   0.0  1.0
..  ...  ...
95  0.0  1.0
96  1.0  0.0
97  1.0  1.0
98  1.0  0.0
99  0.0  0.0

[100 rows x 2 columns]
[0.8 0.8 0.8 0.  0.  0.  0.8 0.8 0.8 0.8 1.  0.8 0.8 0.  0.8 0.  0.8 0.
 0.  0.8 0.8 0.8 0.  0.8 0.8 0.8 0.  0.8 0.8 0.  0.  0.8 0.  0.  0.  0.8
 0.  0.  0.  0.  0.8 0.8 0.  0.2 0.  0.2 0.  0.  0.8 0.8 0.  0.8 0.8 0.2
 1.  0.8 0.  0.8 1.  0.8 0.8 0.8 0.8 0.  0.  1.  0.  0.8 0.8 0.  0.  0.
 0.8 0.  0.8 0.2 0.  0.  0.  0.8 0.  0.  0.  0.  0.8 0.2 0.  0.  0.  0.8
 0.8 0.8 0.  0.  0.2 0.  0.8 1.  0.8 0.2]
[1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0.
 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 1.]


      0    1    2    3    4    5    6    7    8    9
0   0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  1.0
1   1.0  0.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0
2   1.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0
3   0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
4   0.0  0.0  1.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0
..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
95  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
96  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
97  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
98  0.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0
99  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0

[100 rows x 10 columns]
[0.55970684 0.04523249 0.53434497 0.22326625 0.06861262 0.25174314
 0.23066909 0.70828973 0.16162439 0.18254275 0.9958754  0.01587741
 0.12466362 0.07608861 0.50438901 0.29318218 0.20741407 0.3719183
 0.04295126 0.58433253 0.13528845 0.12159657 0.57409157 0.26789578
 0.59643812 0.05593833 0.05536534 0.42813501 0.22889313 0.10779822
 0.23

## Declaring variables determined a-priori

### a) Specfiying Input to Model
- m treatments indexed by t {1,..., m}
- n datapoints indexed by i {(X1, T1, Y1), ..., (Xn, Tn, Yn)} 
- d: depth of decision tree
- n_min: minimum number of datapoints of each treatment in node p
- num_features
- num_cuts

In [17]:
model = gp.Model("Kallus")

m = {0, 1}
n = len(data)
d = 3
n_min = 1
num_features = 2
num_cuts = 2

"""# ---- CONSTRUCTING COMPLETE BINARY TREE ----
# - P = number of nodes in the tree
# - L_c = set of non-leaf nodes
# - L = set of leaf ndoes"""

P = 2**d
L_c = set(range(1, 2**(d-1)))
L = set(range(2**(d-1), P))
print(L)
print(L_c)

# - ancestors: dictionary {leaf nodes: {set of ancestors}}
ancestors = {}
for p in L:
    ancestors[p] = [math.floor(p/(2**j)) for j in range(1, d)]

{4, 5, 6, 7}
{1, 2, 3}


In [18]:
# -- This class is an alternative to solving the right/left ancestor problem --
"""
INPUT: d = depth of tree (which includes root node), so d = 2 would make a tree with {1, 2, 3}
RELEVANT FUNCTIONS:
- get_right_left: For all leaf nodes, returns its right and left ancestors in a dictionary 
                  of {(p, q): 1 or -1 if q is right or left ancestor respectively}
"""
class Tree:
    def __init__(self, d):
        self.depth = d
        self.nodes = list(range(1, 2**(d-1)))
        self.leaves = list(range(2**(d-1), 2**d))
        self.ancestor_rl = {}
    
    def get_left_children(self, n):
        if n in self.nodes:
            return int(2*n)
        else:
            raise Exception ('Invalid node n')
    
    def get_right_children(self, n):
        if n in self.nodes:
            return int(2*n+1)
        else:
            raise Exception ('Invalid node n')
    
    def get_parent(self, n):
        if (n in self.nodes) | (n in self.leaves):
            return int(math.floor(n/2))
        else:
            raise Exception ('Invalide node n')
    
    def get_ancestors(self, direction, n):
        current = n
        ancestors = []
        while current != 1:
            current_buffer = self.get_parent(current)
            if direction == 'r':
                if self.get_right_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            else:
                if self.get_left_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            current = current_buffer
        return ancestors
    
    def get_right_left(self):
        for i in self.leaves:
            right = self.get_ancestors('r', i)
            for j in right:
                self.ancestor_rl[(i, j)] = 1
            left = self.get_ancestors('l', i)
            for j in left:
                self.ancestor_rl[(i, j)] = -1
        return self.ancestor_rl

In [19]:
# Alternative way of retrieving right/left ancestors
tree = Tree(3)
right_left = tree.get_right_left()

print(right_left)

{(4, 2): -1, (4, 1): -1, (5, 2): 1, (5, 1): -1, (6, 1): 1, (6, 3): -1, (7, 3): 1, (7, 1): 1}


In [20]:
"""
- C[p]: finite set of cuts on node p--determined apriori {(l, theta)}
    Representation: dictionary {non-leaf node: list of (l, theta)}
    Require a list instead of set so it's ordered (indexed easily)


From Kallus' paper, he wants us to:
1. For each l in [d], sort data along x_l
2a. For each non-leaf node, pick #features from [d] randomly
2b. Set J = {1, (n-1/#cuts), ..., n-1} in decreasing order of cuts
2c. Set Cp = {(l, midpoint between the two buckets in J) for all dimensions chosen and for all j in J}

Make version of ALG3 to take all features"""


if different_Cp:
    pass
else:
    # -- BINARY COVARIATES --> Create a finite set of cuts for C for all features --
    C = []
    for i in range(len(data.columns)):
        C.append((i, 0))

    print(C)
    
print(C[1][0])

[(0, 0), (1, 0)]
1


In [21]:
""" --- OTHER DATA --- 
BIG M Constraints:
- Ybar
- Ymax
- M

BINARY ENCODING FOR CUTS
- k_p: dictionary {non-leaf node p: k_p value}
- Z_p: dictionary {non-leaf node p: k_p x |C_p| 2d matrix}
"""

# ---- Big M Constraints ----
# Ybar is merged into data, but it could not be. ybar is a numpy array
minimum = min(outcome)

ybar = outcome - minimum
#data['ybar'] = ybar

# Ymax
ymax = max(ybar)

# M
# Find all sums for treatments 1, ..., m
#treatment_counts = treatment.value_counts().to_list()
unique, counts = np.unique(treatment, return_counts=True)
#frequencies = numpy.asarray((unique, counts)).T
M = np.array(counts)
M -= len(L) * n_min
M = max(M)
print(M)


if different_Cp:
# ---- K and Z if we followed the definition of C_p from the paper -----
    for p in L_c:
        k[p] = math.ceil(math.log2(len(C[p])))

    z = {}
    for p in L_c:
        matrix = np.zeros((k[p], len(C[p])))
        print(matrix.shape)
        for i in range(1, k[p]+1):
            for j in range(1, len(C[p])+1):
                if math.floor(j/(2**i)) % 2 == 1: # odd number
                    z[i-1, j-1] = 1
                else:
                    z[i-1, j-1] = 0
        z[p] = matrix

else:
    # ---- K and Z if we had constant C for all nodes ----
    k = math.ceil(math.log2(len(C)))
    z = np.zeros((k, len(C)))
    for i in range(1, k+1):
        for j in range(1, len(C)+1):
            if math.floor(j/(2**i)) % 2 == 1: # odd number
                z[i-1, j-1] = 1
            else:
                z[i-1, j-1] = 0
print(z)

47
[[0. 1.]]


In [22]:
# -- VARIABLE DECLARATION --

# -- Variables to determine: gamma and lambda --
# 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
#       - represent with a matrix gamma (|L_c| x |C_p|)

gamma = model.addVars(L_c, len(C), lb=0, ub=1, name='gamma') 
# This assumes gamma is binary
#gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

# 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
#       - represent with a matrix lamb (|L| x m)
lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


# -- Other Variables in Formulation --
# 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
#       - represent with a matrix w (n x |L|)

w = model.addVars(n, L, lb=0, ub=1, name='w')
# This assumes w is binary, when in reality it is continuous from 0-1
#w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

# 2. mu_p = mean outcome of prescribed treatment in node p
#       - represent with a matrix mu (|L|)
mu = model.addVars(L, lb=0, name='mu') # define in constraint

# 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
#       - represent with a matrix nu (n x |L|)
nu = model.addVars(n, L, lb=0, name='nu')

# 4. delta_p = forces only 1 choice of cut at node p
#       - represent with a dictionary {non-leaf node p: 1d matrix of size k_p}
delta = model.addVars(L, k, vtype=GRB.BINARY, name='delta')

# 5. Chi_i(gamma) = 1 if choice of cut induces datapoint i to go left on the cut gamma, 0 otherwise
chi = model.addVars(L_c, n, vtype=GRB.BINARY, name='chi')

In [23]:
# --- OBJECTIVE FUNCTION ---
model.setObjective(gp.quicksum(nu[i, p] for i in range(n) for p in L), GRB.MINIMIZE)


# --- CONSTRAINTS ---
# Constraint 4c (4b is done by definition of variables)
if delta_include:
    for p in L_c:
        # need to do matrix multiplication somehow, but this might work?
        for j in range(k):
            model.addConstr(delta[p, j] == gp.quicksum(gamma[p, i] * z[j, i] for i in range(len(C))))
            
# Additional constraint that gamma[p] adds up to 1
# CHECKED
for p in L_c:
    model.addConstr(gp.quicksum(gamma[p, i] for i in range(len(C))) == 1)
    
# Add constraint Chi
for i in range(n):
    for p in L_c:
        model.addConstr(chi[p, i] == gp.quicksum(gamma[p, j] for j in range(len(C)) if C[j][1] >= data.iloc[i, C[j][0]]))
        
        
# Constraint 4d&e (Membership restriction from its ancestors) CHECKED
for p in L:
    A_p = ancestors[p] #index ancestors of p
    for q in A_p:
        R_pq = right_left[(p, q)]
        for i in range(n):
            model.addConstr(w[i, p] <= (1+R_pq)/2 - R_pq * chi[q, i])


#4e CHECKED
for p in L:
    A_p = ancestors[p] #index ancestors of p
    for i in range(n):
        model.addConstr(w[i, p] >= 1 + gp.quicksum(-chi[q, i] for q in A_p if right_left[(p, q)] == 1)
                    + gp.quicksum(-1+chi[q, i] for q in A_p if right_left[(p, q)] == -1))
        

# Constraint 4f
# CHECKED
for t in m:
    for p in L:
        model.addConstr(gp.quicksum(w[i, p] for i in range(n) if treatment[i] == t) >= n_min) #assuming the input comes in vector (Xi, Ti, Yi)
        # only add the datapoints that have been given treatment t
        
# Constraints 4g&h (Linearization of nu)
# CHECKED
for p in L:
    for i in range(n):
        model.addConstr(nu[i, p] <= ymax * w[i, p]) # should I calculate Y directly?
        model.addConstr(nu[i, p] <= mu[p])
        model.addConstr(nu[i, p] >= mu[p] - ymax * (1-w[i, p]))

# Constraint 4i (Choice of treatment applied to p)
# CHECKED
for p in L:
    model.addConstr(gp.quicksum(lamb[p, t] for t in m) == 1)
        
# Constraint 4j&k (Consistency between lambda and mu)
# CHECKED. There are some inconsistencies where some w don't appear, but this is because ybar is 0 (i.e. the minimum)
for p in L:
    for t in m:
        model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if treatment[i] == t) <= M*(1-lamb[p, t]))
        model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if treatment[i] == t) >= M*(lamb[p, t]-1))

In [24]:
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2731 rows, 1122 columns and 7869 nonzeros
Model fingerprint: 0x0a74d75f
Variable types: 810 continuous, 312 integer (312 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 2715 rows and 1115 columns
Presolve time: 0.01s
Presolved: 16 rows, 7 columns, 44 nonzeros
Variable types: 5 continuous, 2 integer (2 binary)
Found heuristic solution: objective 43.6000000
Found heuristic solution: objective 39.2000000

Root relaxation: objective 2.317987e+01, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0        39.20000   39.20000  0.00%     -    0s

Explored 0 nodes (5 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 ava

In [25]:
model.printAttr('X')


    Variable            X 
-------------------------
  gamma[1,1]            1 
  gamma[2,0]            1 
  gamma[3,0]            1 
   lamb[4,0]            1 
   lamb[5,1]            1 
   lamb[6,0]            1 
   lamb[7,1]            1 
      w[0,7]            1 
      w[1,7]            1 
      w[2,5]            1 
      w[3,6]            1 
      w[4,6]            1 
      w[5,6]            1 
      w[6,5]            1 
      w[7,5]            1 
      w[8,7]            1 
      w[9,5]            1 
     w[10,7]            1 
     w[11,7]            1 
     w[12,5]            1 
     w[13,4]            1 
     w[14,5]            1 
     w[15,6]            1 
     w[16,7]            1 
     w[17,4]            1 
     w[18,4]            1 
     w[19,7]            1 
     w[20,5]            1 
     w[21,5]            1 
     w[22,6]            1 
     w[23,7]            1 
     w[24,7]            1 
     w[25,7]            1 
     w[26,4]            1 
     w[27,7]            1 
 

   chi[3,93]            1 
   chi[3,94]            1 
   chi[3,95]            1 
   chi[3,99]            1 


In [91]:
print(data)

     0    1
0  0.0  1.0
1  1.0  1.0
2  0.0  1.0
3  1.0  1.0
4  0.0  1.0
5  1.0  0.0
6  1.0  0.0
7  0.0  1.0
8  0.0  0.0
9  1.0  0.0


In [92]:
print (model.display())

Minimize
   <gurobi.LinExpr: nu[0,4] + nu[0,5] + nu[0,6] + nu[0,7] + nu[1,4] + nu[1,5] + nu[1,6] + nu[1,7] + nu[2,4] + nu[2,5] + nu[2,6] + nu[2,7] + nu[3,4] + nu[3,5] + nu[3,6] + nu[3,7] + nu[4,4] + nu[4,5] + nu[4,6] + nu[4,7] + nu[5,4] + nu[5,5] + nu[5,6] + nu[5,7] + nu[6,4] + nu[6,5] + nu[6,6] + nu[6,7] + nu[7,4] + nu[7,5] + nu[7,6] + nu[7,7] + nu[8,4] + nu[8,5] + nu[8,6] + nu[8,7] + nu[9,4] + nu[9,5] + nu[9,6] + nu[9,7]>
Subject To
   R0 : <gurobi.LinExpr: gamma[1,0] + gamma[1,1]> = 1.0
   R1 : <gurobi.LinExpr: gamma[2,0] + gamma[2,1]> = 1.0
   R2 : <gurobi.LinExpr: gamma[3,0] + gamma[3,1]> = 1.0
   R3 : <gurobi.LinExpr: -1.0 gamma[1,0] + chi[1,0]> = 0.0
   R4 : <gurobi.LinExpr: -1.0 gamma[2,0] + chi[2,0]> = 0.0
   R5 : <gurobi.LinExpr: -1.0 gamma[3,0] + chi[3,0]> = 0.0
   R6 : <gurobi.LinExpr: chi[1,1]> = 0.0
   R7 : <gurobi.LinExpr: chi[2,1]> = 0.0
   R8 : <gurobi.LinExpr: chi[3,1]> = 0.0
   R9 : <gurobi.LinExpr: -1.0 gamma[1,0] + chi[1,2]> = 0.0
   R10 : <gurobi.LinExpr: -1.0 gam

In [422]:
for i in data.rows:
    print(i)

AttributeError: 'DataFrame' object has no attribute 'rows'